In [1]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [2]:
import pandas as pd

In [7]:
datagen = pd.read_excel("power_generation.xlsx", sheet_name="gen")

In [8]:
datagen

,id,limit,cost
0,0,20,0.10
1,1,10,0.05
2,2,40,0.30
3,3,50,0.40
4,4,5,0.01


In [5]:
dataload = pd.read_excel("power_generation.xlsx", sheet_name="load")

In [6]:
dataload

,id,value
0,0,50
1,1,20
2,2,30


In [16]:
Ng = len(datagen)

In [17]:
model = pyo.ConcreteModel()

In [18]:
model.Pg = pyo.Var(bounds= (0,None))

In [19]:
model.pprint()

1 Var Declarations
    Pg : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True :  Reals

1 Declarations: Pg


In [20]:
model.Pg = pyo.Var(range(Ng),bounds= (0,None))

'pyomo.core.base.var.ScalarVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [21]:
model.pprint()

1 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    Pg : Size=5, Index=Pg_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals

2 Declarations: Pg_index Pg


In [22]:
Pg =  model.Pg

In [23]:
# constraints

In [24]:
pg_sum = sum([Pg[g] for g in datagen.id])

In [26]:
print(pg_sum)

Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4]


In [27]:
model.balance = pyo.Constraint(expr = pg_sum == sum(dataload.value))

In [28]:
model.cond = pyo.Constraint(expr = Pg[0] + Pg [3] >= dataload.value[0])

In [30]:
model.pprint()

1 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    Pg : Size=5, Index=Pg_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals

2 Constraint Declarations
    balance : Size=1, Index=None, Active=True
        Key  : Lower : Body                                  : Upper : Active
        None : 100.0 : Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4] : 100.0 :   True
    cond : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  50.0 : Pg[0] + Pg[3] :  +Inf :   True

4 Declarations: Pg_index Pg

In [31]:
model.limits = pyo.ConstraintList()

In [32]:
for g in datagen.id:
    model.limits.add(expr = Pg[g] <= datagen.limit[g])

In [33]:
model.pprint()

2 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    limits_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}

1 Var Declarations
    Pg : Size=5, Index=Pg_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals

3 Constraint Declarations
    balance : Size=1, Index=None, Active=True
        Key  : Lower : Body                                  : Upper : Active
        None : 100.0 : Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4] : 100.0 :   True
    cond : Size=1, I

In [34]:
# objective function

In [35]:
model.obj = pyo.Objective(expr = sum([Pg[g]*datagen.cost[g] for g in datagen.id]))

In [36]:
opt = pyo.SolverFactory('glpk')
opt.solve(model) 

{'Problem': [{'Name': 'unknown', 'Lower bound': 25.05, 'Upper bound': 25.05, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 5, 'Number of nonzeros': 12, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.0611116886138916}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [37]:
model.pprint()

2 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    limits_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}

1 Var Declarations
    Pg : Size=5, Index=Pg_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  20.0 :  None : False : False :  Reals
          1 :     0 :  10.0 :  None : False : False :  Reals
          2 :     0 :  35.0 :  None : False : False :  Reals
          3 :     0 :  30.0 :  None : False : False :  Reals
          4 :     0 :   5.0 :  None : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.1*Pg[0] + 0.05*Pg[1] + 0.3*Pg[2] + 0.4*Pg[3] + 0.01*Pg[4]

3 Constraint Declarations
    balance :

In [39]:
[pyo.value(Pg[g]) for g in datagen.id]

[20.0, 10.0, 35.0, 30.0, 5.0]

In [40]:
datagen['Pg'] = [pyo.value(Pg[g]) for g in datagen.id]

In [41]:
datagen

,id,limit,cost,Pg
0,0,20,0.10,20.0
1,1,10,0.05,10.0
2,2,40,0.30,35.0
3,3,50,0.40,30.0
4,4,5,0.01,5.0


In [42]:
sum([pyo.value(Pg[g]) for g in datagen.id])

100.0